In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import ParameterGrid
import random
from collections import Counter

c:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os

if 'COLAB_TPU_ADDR' not in os.environ:
  print('Not connected to TPU')
else:
  print("Connected to TPU")

Not connected to TPU


In [5]:
from google.colab import files
uploaded = files.upload()

Saving a3_train_final.tsv to a3_train_final.tsv
Saving a3_test.tsv to a3_test.tsv


In [2]:
# Read the TSV file
df = pd.read_csv("a3_first_sample.tsv", sep="\t", header=None)

# Add headers "label" and "text"
df.columns = ["label", "text"]

In [4]:
df = df[df['label'] != -1]
df

,label,text
0,0,When a vaccine is safe and effective you don’t...
1,1,Everybody that does not take the vaccine will die
2,0,Well I wont b takin vaccine coz i dont trust it
3,0,I have sickle cell anemia. I'm not taking the ...
4,1,"I’m in the moderna clinical trial, so feel fre..."
...,...,...
970,0,"Thank you for the video, however instead of..."
971,0,Over my dead body will I get the vaccine or tr...
972,0,By CDC's own numbers recovery rates are higher...
973,1,"Alrhough I don't understand everything, at lea..."


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        label = int(self.data.iloc[idx]['label'])  # Convert label to integer
        encoding = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [18]:
# Splitting into train (60%), validation (20%), and test (20%)
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=69)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=69)

# Resetting index for each subset
train_df.reset_index(drop=True, inplace=True)
validation_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [19]:
print(len(train_df))

585


In [20]:
hyperparameters = {
    'learning_rate': [1e-5,2e-5, 3e-5, 5e-5, 4e-4, 1e-4],
    'batch_size': [8, 16,32],
    'num_epochs': [3, 4, 5]
}

# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Iterate over hyperparameter combinations
best_accuracy = 0
best_model = None
best_params = None

for params in ParameterGrid(hyperparameters):
    # Create data loaders
    train_dataset = CustomDataset(train_df)
    val_dataset = CustomDataset(validation_df)
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)

    # Initialize the model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Train the model
    for epoch in range(params['num_epochs']):
        model.train()
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

    # Evaluate the model
    model.eval()
    val_predictions = []
    val_targets = []
    for batch in val_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            val_predictions.extend(preds.cpu().numpy())
            val_targets.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_targets, val_predictions)
    print(f"Validation Accuracy for params {params}: {val_accuracy:.4f}")

    # Check if this model is the best so far
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = model
        best_params = params

# Save the best model
best_model.save_pretrained("best_bert_model")

print("Best Validation Accuracy:", best_accuracy)
print("Best Hyperparameters:", best_params)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Validation Accuracy for params {'batch_size': 8, 'learning_rate': 1e-05, 'num_epochs': 3}: 0.7795


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Validation Accuracy for params {'batch_size': 8, 'learning_rate': 1e-05, 'num_epochs': 4}: 0.7590


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Validation Accuracy for params {'batch_size': 32, 'learning_rate': 0.0001, 'num_epochs': 3}: 0.7590


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Validation Accuracy for params {'batch_size': 32, 'learning_rate': 0.0001, 'num_epochs': 4}: 0.7026


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Validation Accuracy for params {'batch_size': 32, 'learning_rate': 0.0001, 'num_epochs': 5}: 0.7282
Best Validation Accuracy: 0.8358974358974359
Best Hyperparameters: {'batch_size': 16, 'learning_rate': 5e-05, 'num_epochs': 5}
